Example of the process of doing DEMs for all orbits for a given observation of an AR – where we don't window out any intervals because of shifts, etc. Instead, we will save energy-specific information about the percent of NuSTAR emission in the chosen region as a function of time, for later inspection. This will help identify cases where pointing shifts might cause unphysical distortion to the DEM inputs. 

Note: auto-download of an AIA file (for use in the initial co-alignmnet) will break while the JSOC is still down. We will add a thing where you can point to an existing AIA file instead.

Overview:

- Define orbits
- Run time interval selection
- Examine resulting intervals
- Manually establish a co-alignment shift between NuSTAR and AIA
- Automatically find co-alignment shifts + make regions for all other time intervals (note: this relies on the assumption that the COM is a good representation of the location of the brightest source, i.e. that the NuSTAR data is primarially one blob).
- Save AIA region files for NCCS input
- NOT IN THIS NOTEBOOK: YOU THEN TAKE THOSE AND MAKE AIA INPUTS ON THE NCCS
- Conduct AIA/NuSTAR DEMs as a function of time, given all the above
- Plot results.
- Print some stats about "left out" times.

In [1]:
# import matplotlib.pyplot as plt
# import numpy as np
# import glob
# from astropy.io import fits
# from astropy import units as u
# import importlib
# import pathlib

#Path to top-level do-dem directory - edit for your system.
path_to_dodem = '/Users/jmdunca2/do-dem/'
from sys import path as sys_path
sys_path.append(path_to_dodem+'/dodem/')

import nustar_dem_prep as nu
# import initial_analysis as ia
# import orbit_auto as oa
import time_interval_selection as tis
import nustar_utilities as nuutil
# import gauss2D as g2d
import all_nu_analysis as ana
import visualize_dem_results as viz
import images_and_coalignment as iac


import pickle
import pathlib
import importlib
import numpy as np
import subprocess


In [2]:
#rerun all, (hightemp analysis to do rescale)

importlib.reload(ana)

with open('all_targets.pickle', 'rb') as f:
    data = pickle.load(f)

keys = data.keys()

thekey=['29-apr-21']

#for key in list(keys)[8:]:
for key in thekey:
    print(key)
    if key =='22-apr-16_1':
        continue
    ARDict = data[key]
    id_dirs = ARDict['datapaths']
    working_dir = ARDict['working_dir']
    method = ARDict['method']

    ARDict['prepped_aia'] = working_dir+'all_aia_dicts_'+key+'_post/'

    data[key] = ARDict

    with open('all_targets.pickle', 'wb') as f:
             # Pickle the 'data' dictionary using the highest protocol available.
             pickle.dump(data, f, pickle.HIGHEST_PROTOCOL) 

    for id in id_dirs:
        evt_data, hdr = nu.return_submap(datapath=id, fpm='A', return_evt_hdr=True)
        time0, time1 = [nuutil.convert_nustar_time(hdr['TSTART']), nuutil.convert_nustar_time(hdr['TSTOP'])]
        timerange = [time0, time1]
        print(timerange[0].strftime('%H-%M-%S'), timerange[1].strftime('%H-%M-%S'))
    
    ana.do_key_dem(key, plot_xrt=False, method=method, high_temp_analysis=True, rscl=True) #missing_last=True, missing_orbit=1, plot_xrt=False)
    print('')
    print('')


    


29-apr-21
14-53-42 15-54-03
16-30-20 17-30-42
18-06-58 19-07-20
19-43-36 20-43-58
21-20-14 22-16-40
/Users/jmdunca2/do-dem/initial_dem_29apr21/all_aia_dicts_29-apr-21_post//orbit_20615001001/
14-53-45 14-59-45
5.6 7.2
Start Time:  2021-04-29 14:53:45
Stop Time:  2021-04-29 14:59:45

Using inputs from prior DEM run, saved in this file: 
/Users/jmdunca2/do-dem/initial_dem_29apr21/14-53-45_14-59-45/14-53-45_14-59-45_5.6_7.2_29-apr-21_MC_DEM_result.pickle

A94 :     3.44   0.69  20 %
A131 :     29.85   5.97  20 %
A171 :     595.99   119.20  20 %
A193 :     1161.55   232.31  20 %
A211 :     517.94   103.59  20 %
A335 :     26.26   5.25  20 %
Be-thin x2 :     40.74   5.94  15 %
Be-med x2 :     7.29   1.08  15 %
2.5-3.5keV A+B :     6257.20   1251.98  20 %
3.5-6.0keV A+B :     726.24   145.79  20 %
6.0-10.0keV A+B :     3.78   1.18  31 %

Doing 100 iterations of DEMReg with input varied within uncertainty!

Not saving this iteration!!

Rescaled by:  1.180285149776959
5.6 7.1
Start Time:  2021

In [3]:
file = '/Users/jmdunca2/do-dem/initial_dem_29apr21/14-53-45_14-59-45/14-53-45_14-59-45_5.6_7.2_29-apr-21_MC_DEM_result.pickle'

with open(file, 'rb') as f:
    data = pickle.load(f)

data.keys()

dict_keys(['time_interval', 'nuenergies', 'nufpm', 'temp_interval', 'plotMK', 'ts', 'ts_', 'mts', 'trmatrix', 'dn_in', 'edn_in', 'chanax', 'nustar_datapath', 'DEM', 'dn_reg', 'edem', 'edn', 'chisq', 'edn_string', 'fill_color', 'mnrat'])

In [2]:
# importlib.reload(ana)
# key = '26-jul-16_2'

# #===============================================================
# #Update dictionary to have correct aia path for NCCS-prepped data.
# #===============================================================
# with open('all_targets.pickle', 'rb') as f:
#     data = pickle.load(f)

# ARDict = data[key]

# id_dirs = ARDict['datapaths']
# obsids = ARDict['obsids']
# working_dir = ARDict['working_dir']
# method = ARDict['method']

# ARDict['prepped_aia'] = working_dir+'all_aia_dicts_'+key+'_post/'

# data[key] = ARDict

# with open('all_targets.pickle', 'wb') as f:
#          # Pickle the 'data' dictionary using the highest protocol available.
#          pickle.dump(data, f, pickle.HIGHEST_PROTOCOL) 

# #===============================================================
# #DO DEMS


# for id in id_dirs:
#     evt_data, hdr = nu.return_submap(datapath=id, fpm='A', return_evt_hdr=True)
#     time0, time1 = [nuutil.convert_nustar_time(hdr['TSTART']), nuutil.convert_nustar_time(hdr['TSTOP'])]
#     timerange = [time0, time1]
#     print(timerange[0].strftime('%H-%M-%S'), timerange[1].strftime('%H-%M-%S'))

# ana.do_key_dem(key, plot_xrt=False, method=method, high_temp_analysis=True, rscl=True) #missing_last=True, missing_orbit=1, plot_xrt=False)

# #===============================================================




23-26-59 23-36-53

/Users/jmdunca2/do-dem/initial_dem_26jul16_2/region_0/ /Users/jmdunca2/do-dem/initial_dem_26jul16_2/gauss_cen_20201006001_A_user_input_0.reg
23-27-00 23-36-50
5.6 7.2
Start Time:  2016-07-26 23:27:00
Stop Time:  2016-07-26 23:36:50

Using inputs from prior DEM run, saved in this file: 
/Users/jmdunca2/do-dem/initial_dem_26jul16_2/region_0/23-27-00_23-36-50/23-27-00_23-36-50_5.6_7.2_26-jul-16_2_MC_DEM_result.pickle

A94 :     2.24   0.45  20 %
A131 :     14.09   2.82  20 %
A171 :     345.74   69.15  20 %
A193 :     543.05   108.61  20 %
A211 :     191.28   38.26  20 %
A335 :     10.74   2.15  20 %
2.5-3.5keV A+B :     2075.26   415.23  20 %
3.5-6.0keV A+B :     187.94   37.76  20 %
6.0-10.0keV A+B :     0.89   0.31  35 %

Doing 100 iterations of DEMReg with input varied within uncertainty!
Rescaled by:  1.205515818498692
5.6 7.1
Start Time:  2016-07-26 23:27:00
Stop Time:  2016-07-26 23:36:50

Using inputs from prior DEM run, saved in this file: 
/Users/jmdunca2/do-de

In [ ]:
import glob
o=2
prepped_aia_dir=ARDict['prepped_aia']
print(prepped_aia_dir)
orbit_aia_dir = prepped_aia_dir+'/orbit_'+obsids[o]+'/'
print(orbit_aia_dir)
files = glob.glob(orbit_aia_dir+'/*')

for filename in files: 
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    #print(data)
    newdata = data['region0']

    with open(filename, 'wb') as f:
        pickle.dump(newdata, f, pickle.HIGHEST_PROTOCOL)

In [3]:
def move_dems(key, method):


    with open('all_targets.pickle', 'rb') as f:
        data = pickle.load(f)
    
    ARDict = data[key]
    
    id_dirs = ARDict['datapaths']
    obsids = ARDict['obsids']
    working_dir = ARDict['working_dir']
    print(working_dir)

    minT=5.6
    maxT=7.2
    
    if method == 'fit':
        all_time_intervals, all_time_intervals_list = tis.find_all_intervals(working_dir, shush=True, 
                                                                    missing_last=False)
    
        for time in all_time_intervals_list:
            timestring = time[0].strftime('%H-%M-%S')
            stopstring = time[1].strftime('%H-%M-%S')
            timestring=timestring+'_'+stopstring
        
            demfile = working_dir+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
            print(demfile)
    
            status = subprocess.call('cp '+demfile+' ./compact_results/', shell=True) 
    
    
    if method in ['input', 'double']:
        if method=='input':
            region_dirs = iac.find_region_dirs(working_dir)
        if method=='double':
            gauss_stats = ARDict['gauss_stats']
            sep_axis = gauss_stats[0][0]
            region_dirs = iac.find_direction_dirs(working_dir, sep_axis)
            #print(region_dirs)
    
        all_all_time_intervals, fixit, all_all_time_intervals_list = tis.region_time_intervals(region_dirs, id_dirs, shush=True, list_=True) 
    
        for i in range(0, len(region_dirs)):
            atl = all_all_time_intervals_list[i]
            r = region_dirs[i]
    
            for time in atl:
                timestring = time[0].strftime('%H-%M-%S')
                stopstring = time[1].strftime('%H-%M-%S')
                timestring=timestring+'_'+stopstring
            
                demfile = r+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
                demfile_ = demfile.split('/')[-1][0:-7]+'_region_'+str(i)+'.pickle'
                print(demfile_)
        
                status = subprocess.call('cp '+demfile+' ./compact_results/'+demfile_, shell=True) 
        
 

keys = ['01-sep-15', '02-sep-15',
        '19-feb-16', '22-apr-16_2', '26-jul-16_1', '27-jul-16_1', '26-jul-16_2',
        '11-sep-17', '12-sep-17', '13-sep-17', '10-oct-17',
        '29-may-18_1', '09-sep-18', '10-sep-18', 
        '12-apr-19', '13-apr-19', 
        '06-jun-20', '07-jun-20', 
        '08-jun-20', '09-jun-20',
        '29-apr-21', '03-may-21_1', '03-may-21_2', '20-jul-21', 
        '30-jul-21_1', 
        '30-jul-21_2']

with open('all_targets.pickle', 'rb') as f:
    all_targets = pickle.load(f)

for k in keys:
    method = all_targets[k]['method']
    move_dems(k, method)



/Users/jmdunca2/do-dem/initial_dem_1sep15/
03-54-45_03-58-55_5.6_7.2_01-sep-15_MC_DEM_result_region_0.pickle
03-58-55_03-59-25_5.6_7.2_01-sep-15_MC_DEM_result_region_0.pickle
03-59-25_03-59-55_5.6_7.2_01-sep-15_MC_DEM_result_region_0.pickle
03-59-55_04-00-25_5.6_7.2_01-sep-15_MC_DEM_result_region_0.pickle
04-00-25_04-01-15_5.6_7.2_01-sep-15_MC_DEM_result_region_0.pickle
04-01-15_04-04-50_5.6_7.2_01-sep-15_MC_DEM_result_region_0.pickle
04-04-50_04-06-45_5.6_7.2_01-sep-15_MC_DEM_result_region_0.pickle
04-06-45_04-08-50_5.6_7.2_01-sep-15_MC_DEM_result_region_0.pickle
04-08-50_04-10-15_5.6_7.2_01-sep-15_MC_DEM_result_region_0.pickle
04-10-15_04-13-25_5.6_7.2_01-sep-15_MC_DEM_result_region_0.pickle
04-13-25_04-21-25_5.6_7.2_01-sep-15_MC_DEM_result_region_0.pickle
04-21-25_04-47-40_5.6_7.2_01-sep-15_MC_DEM_result_region_0.pickle
08-38-20_08-39-15_5.6_7.2_01-sep-15_MC_DEM_result_region_0.pickle
08-39-15_08-42-05_5.6_7.2_01-sep-15_MC_DEM_result_region_0.pickle
08-42-05_08-46-15_5.6_7.2_01-sep-

/Users/jmdunca2/miniforge3/lib/python3.10/site-packages/astropy/time/formats.py:1580: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  val1_uint32 = val1.view((np.uint32, val1.dtype.itemsize // 4))


19-28-40_19-29-30_5.6_7.2_26-jul-16_1_MC_DEM_result_region_0.pickle
19-29-30_19-31-05_5.6_7.2_26-jul-16_1_MC_DEM_result_region_0.pickle
19-31-05_19-32-15_5.6_7.2_26-jul-16_1_MC_DEM_result_region_0.pickle
19-32-15_19-33-25_5.6_7.2_26-jul-16_1_MC_DEM_result_region_0.pickle
19-33-25_19-34-40_5.6_7.2_26-jul-16_1_MC_DEM_result_region_0.pickle
19-34-40_19-35-45_5.6_7.2_26-jul-16_1_MC_DEM_result_region_0.pickle
19-35-45_19-36-35_5.6_7.2_26-jul-16_1_MC_DEM_result_region_0.pickle
19-36-35_19-37-35_5.6_7.2_26-jul-16_1_MC_DEM_result_region_0.pickle
19-37-35_19-39-55_5.6_7.2_26-jul-16_1_MC_DEM_result_region_0.pickle
19-39-55_19-41-10_5.6_7.2_26-jul-16_1_MC_DEM_result_region_0.pickle
19-41-10_19-44-50_5.6_7.2_26-jul-16_1_MC_DEM_result_region_0.pickle
19-44-50_19-49-05_5.6_7.2_26-jul-16_1_MC_DEM_result_region_0.pickle
19-49-05_19-55-10_5.6_7.2_26-jul-16_1_MC_DEM_result_region_0.pickle
19-55-10_20-01-10_5.6_7.2_26-jul-16_1_MC_DEM_result_region_0.pickle
20-01-10_20-11-35_5.6_7.2_26-jul-16_1_MC_DEM_res

cp: /Users/jmdunca2/do-dem/initial_dem_27jul16/region_0/00-41-10_00-43-25/00-41-10_00-43-25_5.6_7.2_27-jul-16_1_MC_DEM_result.pickle: No such file or directory


00-51-30_00-52-45_5.6_7.2_27-jul-16_1_MC_DEM_result_region_0.pickle
00-52-45_00-54-25_5.6_7.2_27-jul-16_1_MC_DEM_result_region_0.pickle
00-54-25_00-55-50_5.6_7.2_27-jul-16_1_MC_DEM_result_region_0.pickle
00-55-50_00-57-15_5.6_7.2_27-jul-16_1_MC_DEM_result_region_0.pickle
00-57-15_01-00-15_5.6_7.2_27-jul-16_1_MC_DEM_result_region_0.pickle
01-00-15_01-02-15_5.6_7.2_27-jul-16_1_MC_DEM_result_region_0.pickle
01-02-15_01-03-30_5.6_7.2_27-jul-16_1_MC_DEM_result_region_0.pickle
01-03-30_01-04-45_5.6_7.2_27-jul-16_1_MC_DEM_result_region_0.pickle
01-04-45_01-05-40_5.6_7.2_27-jul-16_1_MC_DEM_result_region_0.pickle
01-05-40_01-06-40_5.6_7.2_27-jul-16_1_MC_DEM_result_region_0.pickle
01-06-40_01-07-50_5.6_7.2_27-jul-16_1_MC_DEM_result_region_0.pickle
01-07-50_01-08-55_5.6_7.2_27-jul-16_1_MC_DEM_result_region_0.pickle
01-08-55_01-09-45_5.6_7.2_27-jul-16_1_MC_DEM_result_region_0.pickle
01-09-45_01-10-20_5.6_7.2_27-jul-16_1_MC_DEM_result_region_0.pickle
01-10-20_01-11-20_5.6_7.2_27-jul-16_1_MC_DEM_res

...

cp: /Users/jmdunca2/do-dem/initial_dem_20jul21/north/09-23-00_09-27-50/09-23-00_09-27-50_5.6_7.2_20-jul-21_MC_DEM_result.pickle: No such file or directory


10-08-15_10-13-15_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
10-13-15_10-16-30_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
10-16-30_10-19-05_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
10-19-05_10-19-40_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
10-19-40_10-20-10_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
10-20-10_10-20-40_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
10-20-40_10-21-10_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
10-21-10_10-21-40_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
10-21-40_10-22-10_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
10-22-10_10-22-40_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
10-22-40_10-23-10_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
10-23-10_10-23-40_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
10-23-40_10-24-10_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
10-24-10_10-24-40_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
10-24-40_10-25-10_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
10-25-10_1

cp: /Users/jmdunca2/do-dem/initial_dem_20jul21/north/10-55-15_11-04-25/10-55-15_11-04-25_5.6_7.2_20-jul-21_MC_DEM_result.pickle: No such file or directory


12-08-55_12-09-45_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
12-09-45_12-10-50_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
12-10-50_12-11-40_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
12-11-40_12-12-25_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
12-12-25_12-13-30_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
12-13-30_12-14-05_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
12-14-05_12-14-35_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
12-14-35_12-15-20_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
12-15-20_12-16-15_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
12-16-15_12-17-05_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
12-17-05_12-18-05_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
12-18-05_12-19-10_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
12-19-10_12-20-00_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
12-20-00_12-21-45_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
12-21-45_12-23-05_5.6_7.2_20-jul-21_MC_DEM_result_region_1.pickle
12-23-05_1

cp: /Users/jmdunca2/do-dem/initial_dem_20jul21/north/12-36-30_12-41-05/12-36-30_12-41-05_5.6_7.2_20-jul-21_MC_DEM_result.pickle: No such file or directory


In [ ]:
demfile_

In [ ]:
# importlib.reload(ana)
# key = '30-jul-21_1'
# method='double'

# #===============================================================
# #Update dictionary to have correct aia path for NCCS-prepped data.
# #===============================================================
# with open('all_targets.pickle', 'rb') as f:
#     data = pickle.load(f)

# ARDict = data[key]

# id_dirs = ARDict['datapaths']
# obsids = ARDict['obsids']
# working_dir = ARDict['working_dir']

# ARDict['prepped_aia'] = working_dir+'all_aia_dicts_'+key+'_post/'

# data[key] = ARDict

# with open('all_targets.pickle', 'wb') as f:
#          # Pickle the 'data' dictionary using the highest protocol available.
#          pickle.dump(data, f, pickle.HIGHEST_PROTOCOL) 

# #===============================================================
# #DO DEMS

# import initial_analysis as ia
# import nustar_utilities as nuutil

# for id in id_dirs:
#     evt_data, hdr = nu.return_submap(datapath=id, fpm='A', return_evt_hdr=True)
#     time0, time1 = [nuutil.convert_nustar_time(hdr['TSTART']), nuutil.convert_nustar_time(hdr['TSTOP'])]
#     timerange = [time0, time1]
#     print(timerange[0].strftime('%H-%M-%S'), timerange[1].strftime('%H-%M-%S'))

# ana.do_key_dem(key, plot_xrt=False, method=method) #missing_last=True, missing_orbit=1, plot_xrt=False)

# #===============================================================


# #===============================================================
# #Copy DEM result files to common directory


# minT=5.6
# maxT=7.2

# if method == 'fit':
#     all_time_intervals, all_time_intervals_list = tis.find_all_intervals(working_dir, shush=True, 
#                                                                 missing_last=False)
    
#     for time in all_time_intervals_list:
#         timestring = time[0].strftime('%H-%M-%S')
#         stopstring = time[1].strftime('%H-%M-%S')
#         timestring=timestring+'_'+stopstring
        
#         demfile = working_dir+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
#         print(demfile)
    
#         status = subprocess.call('cp '+demfile+' ./compact_results/', shell=True) 


# if method in ['input', 'double']:
#     if method=='input':
#         region_dirs = iac.find_region_dirs(working_dir)
#     if method=='double':
#         gauss_stats = ARDict['gauss_stats']
#         sep_axis = gauss_stats[0][0]
#         region_dirs = iac.find_direction_dirs(working_dir, sep_axis)
#         #print(region_dirs)

#     all_all_time_intervals, fixit, all_all_time_intervals_list = tis.region_time_intervals(region_dirs, id_dirs, shush=True, list_=True) 

#     for i in range(0, len(region_dirs)):
#         atl = all_all_time_intervals_list[i]
#         r = region_dirs[i]

#         for time in atl:
#             timestring = time[0].strftime('%H-%M-%S')
#             stopstring = time[1].strftime('%H-%M-%S')
#             timestring=timestring+'_'+stopstring
            
#             demfile = r+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
#             demfile_ = demfile[70:-7]+'_region_'+str(i)+'.pickle'
#             print(demfile_)
        
#             status = subprocess.call('cp '+demfile+' ./compact_results/'+demfile_, shell=True) 
    
# #===============================================================



In [ ]:
importlib.reload(ana)
key = '12-sep-17'
method='input'

#===============================================================
#Update dictionary to have correct aia path for NCCS-prepped data.
#===============================================================
with open('all_targets.pickle', 'rb') as f:
    data = pickle.load(f)

ARDict = data[key]

id_dirs = ARDict['datapaths']
obsids = ARDict['obsids']
working_dir = ARDict['working_dir']

ARDict['prepped_aia'] = working_dir+'all_aia_dicts_'+key+'_post/'

data[key] = ARDict

with open('all_targets.pickle', 'wb') as f:
         # Pickle the 'data' dictionary using the highest protocol available.
         pickle.dump(data, f, pickle.HIGHEST_PROTOCOL) 

#===============================================================
#DO DEMS

import nustar_utilities as nuutil

for id in id_dirs:
    evt_data, hdr = nu.return_submap(datapath=id, fpm='A', return_evt_hdr=True)
    time0, time1 = [nuutil.convert_nustar_time(hdr['TSTART']), nuutil.convert_nustar_time(hdr['TSTOP'])]
    timerange = [time0, time1]
    print(timerange[0].strftime('%H-%M-%S'), timerange[1].strftime('%H-%M-%S'))

ana.do_key_dem(key, plot_xrt=False, method=method) #missing_last=True, missing_orbit=1, plot_xrt=False)

#===============================================================


#===============================================================
#Copy DEM result files to common directory


minT=5.6
maxT=7.2

if method == 'fit':
    all_time_intervals, all_time_intervals_list = tis.find_all_intervals(working_dir, shush=True, 
                                                                missing_last=False)
    
    for time in all_time_intervals_list:
        timestring = time[0].strftime('%H-%M-%S')
        stopstring = time[1].strftime('%H-%M-%S')
        timestring=timestring+'_'+stopstring
        
        demfile = working_dir+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
        print(demfile)
    
        status = subprocess.call('cp '+demfile+' ./compact_results/', shell=True) 


if method == 'input':
    region_dirs = iac.find_region_dirs(working_dir)
    all_all_time_intervals, fixit, all_all_time_intervals_list = tis.region_time_intervals(region_dirs, id_dirs, shush=True, list_=True)

    for i in range(0, len(region_dirs)):
        atl = all_all_time_intervals_list[i]
        r = region_dirs[i]

        for time in atl:
            timestring = time[0].strftime('%H-%M-%S')
            stopstring = time[1].strftime('%H-%M-%S')
            timestring=timestring+'_'+stopstring
            
            demfile = r+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
            demfile_ = demfile[70:-7]+'_region_'+str(i)+'.pickle'
            print(demfile_)
        
            status = subprocess.call('cp '+demfile+' ./compact_results/'+demfile_, shell=True) 
    
#===============================================================



In [ ]:
len('/Users/jmdunca2/do-dem/initial_dem_12sep17/region_0/20-56-30_20-57-00/')

In [ ]:
importlib.reload(ana)
key = '09-sep-18'

#===============================================================
#Update dictionary to have correct aia path for NCCS-prepped data.
#===============================================================
with open('all_targets.pickle', 'rb') as f:
    data = pickle.load(f)

ARDict = data[key]

id_dirs = ARDict['datapaths']
obsids = ARDict['obsids']
working_dir = ARDict['working_dir']

ARDict['prepped_aia'] = working_dir+'all_aia_dicts_'+key+'_post/'

data[key] = ARDict

with open('all_targets.pickle', 'wb') as f:
         # Pickle the 'data' dictionary using the highest protocol available.
         pickle.dump(data, f, pickle.HIGHEST_PROTOCOL) 

#===============================================================
#DO DEMS

import initial_analysis as ia
import nustar_utilities as nuutil

for id in id_dirs:
    evt_data, hdr = nu.return_submap(datapath=id, fpm='A', return_evt_hdr=True)
    time0, time1 = [nuutil.convert_nustar_time(hdr['TSTART']), nuutil.convert_nustar_time(hdr['TSTOP'])]
    timerange = [time0, time1]
    print(timerange[0].strftime('%H-%M-%S'), timerange[1].strftime('%H-%M-%S'))

ana.do_key_dem(key, missing_last=True, missing_orbit=1, plot_xrt=False)

#===============================================================


#===============================================================
#Copy DEM result files to common directory


minT=5.6
maxT=7.2

all_time_intervals, all_time_intervals_list = tis.find_all_intervals(working_dir, shush=True, 
                                                                missing_last=False)

for time in all_time_intervals_list:
    timestring = time[0].strftime('%H-%M-%S')
    stopstring = time[1].strftime('%H-%M-%S')
    timestring=timestring+'_'+stopstring
    
    demfile = working_dir+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
    print(demfile)

    status = subprocess.call('cp '+demfile+' ./compact_results/', shell=True) 

#===============================================================



In [ ]:
keys = ['01-sep-15', '02-sep-15',
        '26-jul-16_1', '27-jul-16_1',
        '11-sep-17', '12-sep-17', '13-sep-17',
        '29-may-18_1', '09-sep-18', '10-sep-18', 
        '12-apr-19', '13-apr-19', 
        '06-jun-20', '07-jun-20', 
        '08-jun-20', '09-jun-20',
        '29-apr-21', '03-may-21_1', '03-may-21_2', '20-jul-21', 
        '30-jul-21_1', 
        '30-jul-21_2']

with open('all_targets.pickle', 'rb') as f:
    all_targets = pickle.load(f)
    
done_sources = {k: all_targets[k] for k in keys}
ana.get_exposures(done_sources, dogoes=False)

In [ ]:
import pickle
import numpy as np

aia_file = '/Users/jmdunca2/do-dem/initial_dem_6jun20/all_aia_dicts_06-jun-20_post/orbit_20611003001/20-59-25_20-59-55_aia_prep.pickle'
with open(aia_file, 'rb') as f:
    aia_dat = pickle.load(f)

np.isnan(aia_dat['aia_dn_s_px']).any()

In [ ]:
done_sources = {k: all_targets[k] for k in keys}
ana.get_exposures(done_sources, dogoes=False)

In [ ]:
import pandas as pd

df = pd.read_csv('fpmA.csv')
starts = df['flare_start'].values
stops = df['flare_end'].values

for i in range(0, len(starts)-1):
    print(starts[i], stops[i])

In [ ]:
for f in flaretimes:
    print(f[0], f[1])

In [ ]:
flaretimes=[]
nonflaretimes=[]
for time in all_time_intervals_list:
    b4 = [s < time[0] for s in starts]
    ea = [s > time[0] for s in stops]
    es = np.where(np.logical_and(b4, ea))

    if es[0].size > 0:
        #print(starts[es], stops[es])
        #print(time)
        flaretimes.append(time)
        continue

    b4 = [s > time[0] for s in starts]
    ea = [s < time[1] for s in starts]
    es = np.where(np.logical_and(b4, ea))

    if es[0].size > 0:
        #print(starts[es], stops[es])
        #print(time)
        flaretimes.append(time)
        continue

    nonflaretimes.append(time)

For the time interval to overlap with a flare, it could either ENVELOP a flare, start before (and end during) start during (and end during) and start + end within the flare. 

- fstart tstart tend fend - ENVELOP
- fstart tstart fend tend - STOPSIDE
  
- tstart fstart tend fend - STARTSIDE
- tstart fstart fend tend - DURING



In [ ]:
from matplotlib import pyplot as plt 

logbins = np.geomspace(np.min(all_above10s), np.max(all_above10s), 30)

fig, ax = plt.subplots(figsize=(15,4), tight_layout = {'pad': 1})

ax.hist(all_above10s, bins=logbins, color='skyblue', edgecolor='black')
ax.set_xscale('log')
ax.axvline(1.8e22, color='Red')
ax.axvline(1.5e23, color='Red')
ax.axvspan(1.8e22, 1.5e23, alpha=0.3, color='Red', label='Ishikawa (2017) 95% Interval')
ax.set_ylabel('Number of intervals')
ax.set_xlabel('EM Integrated >10 MK')
ax.legend()

In [ ]:
# import glob

# #Set path to obsid directory - initial pipeline should have been run already.
# ind=0
# datapath=id_dirs[ind]
# obsid=obsids[ind]

# evt_data, hdr = nu.return_submap(datapath=datapath, fpm='A', return_evt_hdr=True)
# time0, time1 = [nuutil.convert_nustar_time(hdr['TSTART']), nuutil.convert_nustar_time(hdr['TSTOP'])]
# timerange = [time0.tt.datetime, time1.tt.datetime]
# from datetime import timezone
# timerange = [t.replace(tzinfo=timezone.utc) for t in timerange]

# #Comment second line if you're not using this same example nustar orbit
# #Edit it to include only the desired time interval (default- all times in file) once you've run this once
# #timerange=[]
# #timerange=[datetime.datetime(2018, 5, 29, 22, 22), datetime.datetime(2018, 5, 29, 23, 20)]

# evtA = glob.glob(datapath+'/event_cl/*A06_cl.evt')
# evtB = glob.glob(datapath+'/event_cl/*B06_cl.evt')
# hkA  = glob.glob(datapath+'/hk/*A_fpm.hk')
# hkB  = glob.glob(datapath+'/hk/*B_fpm.hk')

# import lightcurves as lc

# importlib.reload(lc)
# lc.prepare_nustar_lightcurves(evtA, evtB, hkA, hkB, timebin=15, erange=[2.,4.], 
#                               livetime_corr=False, save_dir=working_dir)
# lc.prepare_nustar_lightcurves(evtA, evtB, hkA, hkB, timebin=15, erange=[4.,6.], 
#                               livetime_corr=False, save_dir=working_dir)
# lc.prepare_nustar_lightcurves(evtA, evtB, hkA, hkB, timebin=15, erange=[6.,10.], 
#                               livetime_corr=False, save_dir=working_dir)

# lc.plot_nustar_lightcurves(eranges = [[2.,4.],[4.,6.],[6.,10.]],
#                            timerange=timerange, save_dir=working_dir)